In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In this Notebook, we will :
- import the different data sets
- clean these data sets
- merge them together 
- put the merged set in an appropriate form for the future sections

## Importing the data

In order to have enough explicative variables that could explain the variation of the temperature from one year to another, we decided to download the data from four different sources :
1) The temperature on Earth since 1743 was download on https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data. This data was already in a `csv` format.

2) The distance to the nearest ocean. Originally, we downloaded a shapefile in https://www.naturalearthdata.com/downloads/10m-physical-vectors/10m-coastline/ and the data presented as a geopandas DataFrame containing coastlines as geometric objects, called `STRING`. From these coastlines, we were able to determine, for each city appearing in 1), the distance to the nearest ocean (c.f. Distance_ocean.ipynb).

3) A csv file containing the elevation of various cities in the world, downloaded on https://github.com/bahar/WorldCityLocations.

4) A csv file containing annual $\mathrm{CO}_2$ emissions per country since 1800 : https://ourworldindata.org/grapher/annual-co2-oil.

In [2]:
## Download the four data sets
##############################
temp_df = pd.read_csv('GlobalLandTemperaturesByCity.csv',parse_dates=['dt'])
ocean_df = pd.read_csv('distance_ocean.csv')
elevation_df = pd.read_csv('elevation.csv')
oil_df = pd.read_csv('annual-co2-oil.csv')

## Cleaning the data

In this subsection, we clean each DataFrame separatly. This includes for example dealing with missing values and checking that each feature has the correct dtype.

### The temperature 

Let's print the top five entries.

In [3]:
temp_df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


From this first observation, we can already do the following transformations :
- Drop the column 'AveragetemperatureUncertainty',
- Transform the columns 'Latitude' and 'Longitude' which are not in an appropriate form for future analysis : It is enough to drop the Letter 'N' and 'E' and similarly for 'S' and 'W' with a minus sign.

It will be also convenient to add extra columns indicating the year and the current month.

In [4]:
## Drop the column 'TemperatureUncertainty'
temp_df.drop(columns=['AverageTemperatureUncertainty'],inplace=True)

## Change Coordinates function
##############################
def ChangeCoord(string):
    ## North or East
    if ('N' in string) or ('E' in string):
        return string[0:-1]
    ## South or West
    elif ('S' in string) or ('W' in string):
        newstring = '-'+string[0:-1]
        return newstring
    
## Transform coordinates columns
################################
temp_df['Latitude'] = temp_df['Latitude'].apply(ChangeCoord)
temp_df['Longitude']= temp_df['Longitude'].apply(ChangeCoord)

## Change the dtype to float numbers
####################################
temp_df['Longitude'] = temp_df['Longitude'].astype(np.float)
temp_df['Latitude'] = temp_df['Latitude'].astype(np.float)

## Function returning month/year
def get_month(time):
    return time.month
def get_year(time):
    return time.year

## Applying these functions and create new columns
temp_df['Month'] = temp_df['dt'].apply(get_month)
temp_df['Year'] = temp_df['dt'].apply(get_year)

In [5]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599212 entries, 0 to 8599211
Data columns (total 8 columns):
dt                    datetime64[ns]
AverageTemperature    float64
City                  object
Country               object
Latitude              float64
Longitude             float64
Month                 int64
Year                  int64
dtypes: datetime64[ns](1), float64(3), int64(2), object(2)
memory usage: 524.9+ MB


Let's now have a look at the missing values.

In [6]:
temp_df.isnull().sum()

dt                         0
AverageTemperature    364130
City                       0
Country                    0
Latitude                   0
Longitude                  0
Month                      0
Year                       0
dtype: int64

There are more than $360000$ missing values, which represents less than $5$% of the total observations. We decide to just drop these missing observations !

In [7]:
## Drop NaN values
temp_df = temp_df.dropna()

### Adding the continent

In the Notebook devoted to the data analysis, it will be convenient to have the continent for each country. To achieve this, we download the following excel file :

In [8]:
continent_df = pd.read_excel('Contient.xlsx')

In [9]:
continent_df.head()

,Afghanistan,AFG,4,Southern Asia,Unnamed: 4,Asia,Unnamed: 6
0,2.0,Åland Islands,ALA,248.0,Northern Europe,NaN,Europe
1,3.0,Albania,ALB,8.0,Southern Europe,NaN,Europe
2,4.0,Algeria,DZA,12.0,Northern Africa,NaN,Africa
3,5.0,American Samoa,ASM,16.0,Polynesia,NaN,Oceania
4,6.0,Andorra,AND,20.0,Southern Europe,NaN,Europe


Let's just rename and select the two columns we need

In [10]:
## Select and rename
continent_df = continent_df.iloc[:,[1,6]].rename(
                        columns={'AFG':'Country','Unnamed: 6':'Continent'})
## Add the first line
continent_df = continent_df.append({'Country':'Afghanistan',
                                    'Continent':'Asia'},ignore_index=True)

To be sure, we need to check that the countries of our data set match exactly the above list of countries.

In [11]:
countries = set(temp_df.Country.value_counts().index.values)
countries1 = set(continent_df.Country.values)
## The list of different countries
list(countries-countries1)

["Côte D'Ivoire"]

Fortunately, there is only one country to correct. We can find it using the pandas string funtion `.contains`

In [12]:
continent_df[continent_df.Country.str.contains('Côte')]

,Country,Continent
55,Côte Ivoire,Africa


In [13]:
## Replace in temp_df
temp_df['Country'] = temp_df['Country'].str.replace("Côte D'Ivoire",
                                                        'Côte Ivoire')

In [14]:
## Merge the two dada set together with respect to the country column
temp_df = pd.merge(temp_df,continent_df,on='Country')

In [15]:
temp_df.head()

,dt,AverageTemperature,City,Country,Latitude,Longitude,Month,Year,Continent
0,1743-11-01,6.068,Århus,Denmark,57.05,10.33,11,1743,Europe
1,1744-04-01,5.788,Århus,Denmark,57.05,10.33,4,1744,Europe
2,1744-05-01,10.644,Århus,Denmark,57.05,10.33,5,1744,Europe
3,1744-06-01,14.051,Århus,Denmark,57.05,10.33,6,1744,Europe
4,1744-07-01,16.082,Århus,Denmark,57.05,10.33,7,1744,Europe


### The distance to the nearest ocean

In [16]:
ocean_df.head()

,City,Distance to ocean
0,A Coruña,4084.890091
1,Aachen,162610.022319
2,Aalborg,2598.928645
3,Aba,64247.212994
4,Abadan,4301.465695


In [17]:
ocean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3448 entries, 0 to 3447
Data columns (total 2 columns):
City                 3448 non-null object
Distance to ocean    3448 non-null float64
dtypes: float64(1), object(1)
memory usage: 54.0+ KB


There is nothing to do here since the features have already the right dtype and there are no missing values.

### The elevation

In [18]:
elevation_df.head()

,Country,City,Altitude
0,Afghanistan,Kandahar,1015.0
1,Afghanistan,Mazar-e Sharif,369.0
2,Afghanistan,Herat,927.0
3,Afghanistan,Jalalabad,573.0
4,Afghanistan,Konduz,394.0


In [19]:
elevation_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10567 entries, 0 to 10566
Data columns (total 3 columns):
Country     10567 non-null object
City        10563 non-null object
Altitude    10567 non-null float64
dtypes: float64(1), object(2)
memory usage: 247.7+ KB


Again, there is almost nothing to do here since there are only $4$ NaN values in the column 'City' and the features already have the correct dtype.

In [20]:
elevation_df = elevation_df.dropna()

### Oil consumption

In [21]:
oil_df.head()

,Entity,Code,Year,Oil (CDIAC & GCP (2020))
0,Afghanistan,AFG,1949,0.000000
1,Afghanistan,AFG,1950,0.065952
2,Afghanistan,AFG,1951,0.065952
3,Afghanistan,AFG,1952,0.062288
4,Afghanistan,AFG,1953,0.065952


In [22]:
oil_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17742 entries, 0 to 17741
Data columns (total 4 columns):
Entity                      17742 non-null object
Code                        17742 non-null object
Year                        17742 non-null int64
Oil (CDIAC & GCP (2020))    17742 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 554.5+ KB


The last column represents the quantity of $\mathrm{CO}_2$ emissions from oil in million tonnes. Here we just drop the feature 'Code' which is useless and we rename the column 'Entity' and the last one.

In [23]:
oil_df.drop(columns=['Code'],inplace=True)
oil_df.rename(columns={'Entity':'Country',
'Oil (CDIAC & GCP (2020))':'CO2'},
                              inplace=True)

In [24]:
oil_df.head()

,Country,Year,CO2
0,Afghanistan,1949,0.000000
1,Afghanistan,1950,0.065952
2,Afghanistan,1951,0.065952
3,Afghanistan,1952,0.062288
4,Afghanistan,1953,0.065952


In [25]:
## Take cumulative sum
list_countries = list(oil_df.Country.value_counts().index)
DataFrames = []
for country in list_countries:
    filter_df = oil_df[oil_df.Country == country]
    filter_df=filter_df.sort_values(by='Year')
    filter_df['cumul(CO2)'] = np.cumsum(filter_df.CO2.values)
    DataFrames.append(filter_df)
oil_df = pd.concat(DataFrames)

## Merging the data

In this section, we merge together the four DataFrames in one data set. The temperature and the distance to the ocean are merged with respect to the 'City' column while the $\mathrm{CO}_2$ emission is merged on the 'Country' and the 'Year' features and the elevation is with respect to the 'City' and the 'Country'. The main disadvantage of this process is that we will loose several data points because the different lists of cities do not match each other perfectly. This is the price we pay to have more explicative variables.

In [26]:
## Temperature + ocean distance
df = pd.merge(temp_df,ocean_df,on='City')
## Add the CO2 emission
df = pd.merge(df,oil_df,on=['Country','Year'])
## Add the elevation
df = pd.merge(df,elevation_df,on=['City','Country'])

In [27]:
df.head()

,dt,AverageTemperature,City,Country,Latitude,Longitude,Month,Year,Continent,Distance to ocean,CO2,cumul(CO2),Altitude
0,1843-01-01,1.427,Aalborg,Denmark,57.05,10.33,1,1843,Europe,2598.928645,0.0,0.0,1.0
1,1843-02-01,-0.687,Aalborg,Denmark,57.05,10.33,2,1843,Europe,2598.928645,0.0,0.0,1.0
2,1843-03-01,0.888,Aalborg,Denmark,57.05,10.33,3,1843,Europe,2598.928645,0.0,0.0,1.0
3,1843-04-01,5.516,Aalborg,Denmark,57.05,10.33,4,1843,Europe,2598.928645,0.0,0.0,1.0
4,1843-05-01,9.589,Aalborg,Denmark,57.05,10.33,5,1843,Europe,2598.928645,0.0,0.0,1.0


## Transforming the data

We will produce here two different DataFrames. The first one will be used for the Ridge regression and the $k$-NN model. The data set will be the same as the previous `df`, except that the absolute temperatures are replaced by annual temperatures differences. The second data set will be used for the $k$-mean algorithm. Here the rows will correspond to the cities and the annual temperatures differences will be part of the features.

### Get annual temperature differences

In [28]:
## The list of all cities
cities_list = df.groupby('City').mean().index.values
## The column's name
cols = list(df.columns.values)
## Remove country and city which desappear when resample
cols.remove('City')
cols.remove('Country')
cols.remove('Continent')

In [29]:
DataFrames = []
## Loop over the cities
for city in cities_list:
    ## Filter df 
    frame = df[df['City']== city]
    country = frame.iloc[0,3]
    continent = frame.iloc[0,8]
    ## Resample with montly frequency and aggregate using the mean()
    frame.set_index('dt',inplace=True)
    frame = frame.resample('MS').agg(['mean'])
    frame.reset_index(inplace=True)
    ## Re-order the columns
    frame.columns = cols
    frame['City']=[city]*len(frame)
    frame['Country'] = [country]*len(frame)
    frame['Continent']=[continent]*len(frame)
    ## Compute annual differences
    frame['Annual Temperature Difference'] = (frame['AverageTemperature']-
                        frame['AverageTemperature'].shift(periods=12))
    ## Drop the missing values created from the resample() step
    frame = frame.dropna()
    ## Add this frame to the list
    DataFrames.append(frame)

## Concatenate all the frames
df = pd.concat(DataFrames)

In [30]:
df.drop(columns='AverageTemperature',inplace=True)

In [31]:
## Do the same process for the original DataFrame
#################################################
cities_list1 = temp_df.groupby('City').mean().index.values
cols = list(temp_df.columns.values)
cols.remove('City')
cols.remove('Country')
cols.remove('Continent')
DataFrames=[]
## Loop over the cities
for city in cities_list1:
    ## Filter df 
    frame = temp_df[temp_df['City']== city]
    country = frame.iloc[0,3]
    continent = frame.iloc[0,8]
    ## Resample with montly frequency and aggregate using the mean()
    frame.set_index('dt',inplace=True)
    frame = frame.resample('MS').agg(['mean'])
    frame.reset_index(inplace=True)
    ## Re-order the columns
    frame.columns = cols
    frame['City']=[city]*len(frame)
    frame['Country'] = [country]*len(frame)
    frame['Continent']=[continent]*len(frame)
    ## Compute annual differences
    frame['Annual Temperature Difference'] = (frame['AverageTemperature']-
                        frame['AverageTemperature'].shift(periods=12))
    ## Drop the missing values created from the resample() step
    frame = frame.dropna()
    ## Add this frame to the list
    DataFrames.append(frame)

## Concatenate the different frames
temp_df = pd.concat(DataFrames)

In [32]:
temp_df.drop(columns='AverageTemperature',inplace=True)

### Preparation for the $k$-mean algorithm

The goal here is to transform the DataFrame in such a way that the rows correspond to cities. As features, we will keep the coordinates, year, month and we will produce additional columns for the annual temperature differences using the `.unstack()` method from pandas.

In [33]:
## columns to use
columns = ['Latitude','Longitude','Month','Year','City',
                           'Annual Temperature Difference']
## Keep track of the coordinates
df_coordinates = temp_df.groupby('City').mean()[['Latitude','Longitude']]
df_coordinates.reset_index(inplace=True)

## Group the cities 
df_kmean = temp_df[columns].groupby(['City','Year','Month']).mean()
df_kmean.head()

Latitude  Longitude  Annual Temperature Difference
City     Year   Month                                                    
A Coruña 1744.0 11.0      42.59      -8.73                          0.249
         1745.0 4.0       42.59      -8.73                         -1.784
         1751.0 1.0       42.59      -8.73                          0.430
                2.0       42.59      -8.73                         -2.821
                3.0       42.59      -8.73                         -0.066

In [34]:
## Apply the unstack method
df_kmean = df_kmean.unstack(level=1).unstack()

## Keep only the temperatures
df_kmean = df_kmean['Annual Temperature Difference']
df_kmean.head()

Year     1744.0                                               ... 2013.0  \
Month      1.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  9.0  10.0  ...   3.0    
City                                                          ...          
A Coruña    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ... -2.240   
Aachen      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ... -6.326   
Aalborg     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ... -6.524   
Aba         NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ... -0.382   
Abadan      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  1.361   

Year                                                             
Month      4.0    5.0    6.0    7.0    8.0  9.0  10.0 11.0 12.0  
City                                                             
A Coruña  1.655 -2.435 -0.156  2.741  1.437  NaN  NaN  NaN  NaN  
Aachen    0.374 -3.045  0.488  2.523 -0.861  NaN  NaN  NaN  NaN  
Aalborg  -0.852 -0.044  1.894  1.513  0.417  NaN  NaN  NaN  NaN  
Aba      -0.624 -0.026 -0.067  0.456 -0.432  NaN  NaN  NaN  NaN  
Abadan   -1.233 -2.872 -1.113 -0.530 -1.422  NaN  NaN  NaN  NaN  

[5 rows x 3180 columns]

In [35]:
## Fill missing values with the previous valid delta T
df_kmean = df_kmean.fillna(method='ffill',axis=1)
df_kmean = df_kmean.fillna(value=0)
## Reset index
df_kmean.reset_index(inplace=True)
## Add the coordinates
df_kmean = pd.merge(df_coordinates,df_kmean,on='City')

/Users/raphaelzacharias/anaconda3/envs/exts-ml/lib/python3.6/site-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/raphaelzacharias/anaconda3/envs/exts-ml/lib/python3.6/site-packages/pandas/core/generic.py:3812: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


In [36]:
df_kmean.head()

,City,Latitude,Longitude,"(1744.0, 1.0)","(1744.0, 2.0)","(1744.0, 3.0)","(1744.0, 4.0)","(1744.0, 5.0)","(1744.0, 6.0)","(1744.0, 7.0)",...,"(2013.0, 3.0)","(2013.0, 4.0)","(2013.0, 5.0)","(2013.0, 6.0)","(2013.0, 7.0)","(2013.0, 8.0)","(2013.0, 9.0)","(2013.0, 10.0)","(2013.0, 11.0)","(2013.0, 12.0)"
0,A Coruña,42.59,-8.73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.240,1.655,-2.435,-0.156,2.741,1.437,1.437,1.437,1.437,1.437
1,Aachen,50.63,6.34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-6.326,0.374,-3.045,0.488,2.523,-0.861,-0.861,-0.861,-0.861,-0.861
2,Aalborg,57.05,10.33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-6.524,-0.852,-0.044,1.894,1.513,0.417,0.417,0.417,0.417,0.417
3,Aba,5.63,8.07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.382,-0.624,-0.026,-0.067,0.456,-0.432,-0.432,-0.432,-0.432,-0.432
4,Abadan,29.74,48.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.361,-1.233,-2.872,-1.113,-0.530,-1.422,-1.422,-1.422,-1.422,-1.422


In [37]:
## Rename the Annual Temperature difference by Delta T
df.rename(columns = {'Annual Temperature Difference':'$\Delta$T'}
          ,inplace=True)
temp_df.rename(columns = {'Annual Temperature Difference':'$\Delta$T'}
          ,inplace=True)
df_kmean.rename(columns = {'Annual Temperature Difference':'$\Delta$T'}
          ,inplace=True)

In [38]:
## Create the csv files
df.to_csv('data_extra_f.csv',index=False, encoding='utf-8')
df_kmean.to_csv('data_kmean.csv',index=False, encoding='utf-8')
temp_df.to_csv('data_full.csv',index=False,encoding='utf-8')